<a href="https://colab.research.google.com/github/sanakashgouli/NRC-Project/blob/main/Full_Pipeline_SOKEGraph_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing requirements**

In [1]:
!pip install openai --upgrade PyPDF2 py2neo requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: openai
    Found existing installation: openai 1.84.0
    Uninstalling openai-1.84.0:
      Successfully uninstalled openai-1.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.


In [2]:
import os
import zipfile
import json
import re
import requests
import openai
from openai import OpenAI
from PyPDF2 import PdfReader
from collections import defaultdict
from py2neo import Graph, Node, Relationship

In [3]:
pip install semanticscholar

In [4]:
pip install google-generativeai

**Abstract Extraction, OpenAI Model Integration, and Knowledge Graph Pipeline**

In [5]:
import google.generativeai as genai
from itertools import cycle

# ✅ Load and rotate Gemini API keys from APIs.txt
with open("APIs.txt") as f:
    api_keys = [line.strip() for line in f if line.strip()]
key_cycle = cycle(api_keys)

def configure_gemini():
    key = next(key_cycle)
    genai.configure(api_key=key)
    return genai.GenerativeModel("gemini-2.0-flash")


In [ ]:
import requests
from semanticscholar import SemanticScholar
import pandas as pd
from time import sleep

# Initialize API
sch = SemanticScholar()

# Define your queries
search_queries = [
    "Acidic earth abundant catalysts for water splitting"
]

# Global limit
max_total = 200
semantic_scholar_papers = []

# Fetch papers
for query in search_queries:
    if len(semantic_scholar_papers) >= max_total:
        break

    print(f"🔍 Searching Semantic Scholar for: {query}")
    try:
        results = sch.search_paper(query, limit=100)  # Use a higher limit per query for efficiency
        for paper in results:
            if len(semantic_scholar_papers) >= max_total:
                break
            semantic_scholar_papers.append({
                "paper_id": paper.paperId,
                "title": paper.title,
                "abstract": paper.abstract or "",
                "authors": ", ".join([a["name"] for a in paper.authors]) if paper.authors else "",
                "year": paper.year,
                "venue": paper.venue,
                "url": paper.url,
                "doi": paper.externalIds.get("DOI") if paper.externalIds else ""
            })
        sleep(1)
    except Exception as e:
        print(f"❌ Error searching '{query}': {e}")
        continue

# Deduplicate
unique_papers = {p['paper_id']: p for p in semantic_scholar_papers}
semantic_scholar_papers = list(unique_papers.values())

print(f"✅ Retrieved {len(semantic_scholar_papers)} unique papers.")


# ✅ Step 2: Organize metadata using title as the paper ID (truncated if needed)
def safe_title(title, max_len=100):
    # Remove unsafe filename characters and truncate
    return re.sub(r'[^a-zA-Z0-9_\- ]', '', title)[:max_len].strip()

text_data = {}
title_map = {}
abstract_map = {}

for paper in semantic_scholar_papers:
    safe_id = safe_title(paper['title'] or paper['paper_id'])
    text_data[safe_id] = paper['abstract'] or ""
    title_map[safe_id] = paper['title'] or ""
    abstract_map[safe_id] = paper['abstract'] or ""

print(f"✅ Organized {len(text_data)} papers with title-based IDs.")



# Step 3: Load ontology
if not os.path.exists("ontology.json"):
    print("⚠️ 'ontology.json' not found. Using built-in fallback.")

try:
    ontology = json.load(open("ontology.json"))
except:
    ontology = {
        "Environment": {
            "Acidic": ["pH < 7", "pH<7", "pH<", "pH 1", "pH=1", "acidic"],
            "Alkaline": ["pH > 7", "pH>7", "pH>", "pH=14", "alkaline", "basic"],
            "Neutral": ["pH 7", "neutral pH"],
            "Operating Temperature": ["temperature", "25°C", "80°C", "temperature-controlled"],
            "Pressure": ["1 atm", "10 atm", "high-pressure", "pressure"],
            "Gas-Phase Reactions": ["gas-phase", "gas reaction"],
            "Non-aqueous Solvents": ["non-aqueous", "organic solvent", "ethanol"]
        },
        "Process": {
            "Electrochemical Energy Conversion": ["electrochemical cell", "redox energy", "electrochemical process"],
            "Water Electrolysis": ["electrolysis of water", "splitting H2O", "hydrogen from water"],
            "Photoelectrochemical Water Splitting": ["photoelectrochemical", "PEC water splitting", "solar water splitting"],
            "Photocatalysis": ["photocatalyst", "UV-activated", "light-driven catalysis"],
            "Thermochemical Water Splitting": ["high-temperature splitting", "thermal decomposition of water"],
            "Electrocatalysis": ["electrocatalyst", "electrocatalytic", "electrode reaction enhancement"],
            "Fuel Cells": ["fuel cell", "proton exchange membrane", "PEM", "electricity generation from fuel"]
        },
        "Reaction": {
            "Hydrogen Evolution Reaction (HER)": ["hydrogen evolution", "HER", "H2 generation", "reduction of protons"],
            "Oxygen Evolution Reaction (OER)": ["oxygen evolution", "OER", "oxygen generation", "water oxidation"],
            "Proton-Coupled Electron Transfer (PCET)": ["PCET", "proton-electron", "concerted proton electron transfer"]
        },
        "Elemental Composition": {
            "Iron": ["Fe", "iron-based catalyst", "Fe3+", "Fe2+"],
            "Nickel": ["Ni", "nickel foam", "nickel oxide"],
            "Cobalt": ["Co", "Co3O4", "cobalt-doped"],
            "Manganese": ["Mn", "manganese oxide"],
            "Molybdenum": ["Mo", "MoS2", "molybdenum disulfide"],
            "Transition Metals": ["transition metal", "d-block metal", "first-row metals"],
            "Earth-Abundant Elements": ["earth-abundant", "non-precious", "low-cost metals"],
            "Metal-Free Catalysts": ["carbon-based", "N-doped graphene", "non-metal"],
            "Dopants": ["doped with P", "S-doping", "boron doping"]
        },
        "Material": {
            "Metal Oxides": ["Fe2O3", "NiO", "oxide layer", "metal oxide"],
            "Metal Phosphides": ["Ni2P", "FeP", "metal phosphide catalyst"],
            "Sulfides, Nitrides, Carbides": ["MoS2", "nitrides", "carbides", "WS2"],
            "Core–Shell Nanostructures": ["core-shell", "encapsulated nanoparticles"],
            "2D Materials": ["graphene", "MoS2", "nanosheets"],
            "Porous Structures, Nanorods, Nanowires": ["nanorods", "nanowires", "porous", "mesoporous", "nanoporous"]
        },
        "Performance & Stability": {
            "Overpotential": ["overpotential of", "η = ", "potential above thermodynamic"],
            "Tafel Slope": ["Tafel slope", "mV dec⁻¹", "Tafel plot"],
            "Exchange Current Density": ["exchange current density", "j0", "baseline current"],
            "Stability": ["chronoamperometry", "cycling stability", "durability test", "long-term performance"]
        },
        "Application": {
            "Green Hydrogen Economy": ["green hydrogen", "H2 economy", "renewable hydrogen"],
            "Hydrogen Production": ["H2 production", "hydrogen generation"],
            "Renewable Energy Integration": ["solar + water splitting", "renewable system", "wind-driven electrolysis"],
            "PEM Electrolyzer": ["proton exchange membrane", "PEM electrolyzer", "membrane cell"],
            "Lithium-Ion Battery": ["LIB", "lithium battery", "anode", "cathode"]
        }
    }

def load_api_keys(file='APIs.txt'):
    with open(file, 'r') as f:
        return [line.strip() for line in f if line.strip()]
api_keys = load_api_keys()
api_index = 0

def get_next_api_key():
    global api_index
    key = api_keys[api_index]
    api_index = (api_index + 1) % len(api_keys)
    return key

print("Total API keys loaded:", len(api_keys))

# Step 4: Call OpenAI Flash model

def call_openai_model(layer_name, abstract_text, ontology_layer):
    """
    Calls OpenAI Flash model to extract keywords and metadata from abstract text for a specific ontology layer.
    """
    model = configure_gemini()

    prompt = f'''
You are a structured information extraction AI specialized in materials science.

You will extract keywords relevant to the layer: **{layer_name}**
For this layer, the following categories and sample keywords are provided:
{json.dumps(ontology_layer, indent=2)}

From the text below, extract:
- Any new or related keywords
- The category it fits under
- Any nearby numeric values with units (e.g., overpotential, pH, current density)

Text to analyze:
{abstract_text}

Return the output in this exact JSON list format:
[
  {{
    "layer": "LayerName",
    "matched_category": "Category",
    "keyword": "ExtractedKeyword",
    "meta_data": "Text snippet"
  }},
  ...
]
'''

    try:
        response = model.generate_content(prompt)
        model_output = response.text.strip()

        print("📄 Gemini returned:\n", model_output[:500])  # Show preview only

        try:
            parsed = json.loads(model_output)
            print(f"✅ JSON parsed successfully. Extracted {len(parsed)} items.")
            return parsed
        except json.JSONDecodeError:
            print("⚠️ JSON parsing failed. Attempting fallback regex...")

            pattern = r'"layer":\s*"([^"]+)",\s*"matched_category":\s*"([^"]+)",\s*"keyword":\s*"([^"]+)",\s*"meta_data":\s*"([^"]+)"'
            matches = re.findall(pattern, model_output, re.DOTALL)
            extracted = []
            for m in matches:
                extracted.append({
                    "layer": m[0].strip(),
                    "matched_category": m[1].strip(),
                    "keyword": m[2].strip(),
                    "meta_data": m[3].strip()
                })

            print(f"✅ Extracted {len(extracted)} items via regex.")
            return extracted

    except Exception as e:
        print(f"❌ OpenAI API call failed: {e}")
        return []



# Step 5: Extract keywords for each layer
ontology_extractions = defaultdict(lambda: defaultdict(list))

for paper_id, abstract in text_data.items():
    print(f"🔍 Processing paper: {paper_id}")
    for layer, categories in ontology.items():
        results = call_openai_model(layer, abstract, categories)
        for res in results:
            cat = res["matched_category"]
            if cat not in ontology[layer]:
                print(f"⚠️ Skipping unknown category '{cat}' under layer '{layer}'")
                continue
            keywords = list(set(ontology[layer][cat] + [res["keyword"]]))
            ontology_extractions[layer][cat].append({
                "keywords": keywords,
                "meta_data": res["meta_data"],
                "paper_id": paper_id
            })
        print(f"✅ Total items extracted for paper {paper_id}, layer '{layer}': {len(results)}")


# Step 6: Parse numeric meta data
def parse_meta_data(meta_str):
    # Normalize formatting
    meta_str = meta_str.replace("cm\u22122", "cm^-2")
    meta_str = meta_str.replace("−", "-").replace("µ", "u").replace("μ", "u")

    # Pattern for numbers + scientific units
    pattern = r"([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?)\s*([a-zA-Zμµ°²^/%\-]+)"
    return [{"value": float(val), "unit": unit} for val, unit in re.findall(pattern, meta_str)]

# Apply parsing to all items in ontology_extractions
for layer, cats in ontology_extractions.items():
    for cat, items in cats.items():
        for item in items:
            item["parsed_meta"] = parse_meta_data(item["meta_data"])

            # 🧪 Tag overpotential or Tafel slope explicitly
            for parsed in item["parsed_meta"]:
                meta_lower = item["meta_data"].lower()
                if "overpotential" in meta_lower:
                    item["overpotential"] = parsed
                if "tafel" in meta_lower:
                    item["tafel_slope"] = parsed

🔍 Searching Semantic Scholar for: Acidic earth abundant catalysts for water splitting
✅ Retrieved 200 unique papers.
✅ Organized 199 papers with title-based IDs.
⚠️ 'ontology.json' not found. Using built-in fallback.
Total API keys loaded: 20
🔍 Processing paper: Invited Atomic Layer Deposited Sulfides As PreCatalysts for Electrochemical Water Splitting
📄 Gemini returned:
 ```json
[
  {
    "layer": "Environment",
    "matched_category": "Acidic",
    "keyword": "acidic environment",
    "meta_data": "For proton exchange membrane electrolyzers featuring an acidic environment, platinum group metals are currently used."
  },
  {
    "layer": "Environment",
    "matched_category": "Acidic",
    "keyword": "0.5 M H2SO4",
    "meta_data": "Electrochemical measurements in 0.5 M H2SO4 combined with ex situ and quasi in situ XPS revealed a-MoSx films to transform into hig
⚠️ JSON parsing failed. Attempting fallback regex...
✅ Extracted 4 items via regex.
✅ Total items extracted for paper Invite

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 306.07ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 306.37ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 306.22ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 307.12ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 281.95ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 308.51ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 306.62ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generat

❌ OpenAI API call failed: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
✅ Total items extracted for paper Invited Atomic Layer Deposited Sulfides As PreCatalysts for Electrochemical Water Splitting, layer 'Process': 0
❌ OpenAI API call failed: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
✅ Total items extracted for paper Invited Atomic Layer Deposited Sulfides As PreCatalysts for Electrochemical Water Splitting, layer 'Reaction': 0


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 307.63ms


📄 Gemini returned:
 ```json
[
  {
    "layer": "Elemental Composition",
    "matched_category": "Molybdenum",
    "keyword": "MoS2",
    "meta_data": "molybdenum sulfides in both amorphous (a-MoSx) and crystalline (c-MoS2) forms have attracted attention"
  },
  {
    "layer": "Elemental Composition",
    "matched_category": "Molybdenum",
    "keyword": "MoSx",
    "meta_data": "amorphous MoSx with adjustable S/Mo ratio (2.8–4.7) and crystalline MoS2"
  },
  {
    "layer": "Elemental Composition",
    "matched_categ
⚠️ JSON parsing failed. Attempting fallback regex...
✅ Extracted 11 items via regex.
✅ Total items extracted for paper Invited Atomic Layer Deposited Sulfides As PreCatalysts for Electrochemical Water Splitting, layer 'Elemental Composition': 11
📄 Gemini returned:
 ```json
[
  {
    "layer": "Material",
    "matched_category": "Sulfides, Nitrides, Carbides",
    "keyword": "molybdenum sulfides",
    "meta_data": "Among these, molybdenum sulfides in both amorphous (a-MoSx) an

...

❌ OpenAI API call failed: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
✅ Total items extracted for paper Earth-Abundant Oxygen Evolution Catalysts Coupled onto ZnO Nanowire Arrays for Efficient Photoelectr, layer 'Process': 0
📄 Gemini returned:
 ```json
[
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (OER)",
    "keyword": "water oxidation",
    "meta_data": "kinetics of water oxidation"
  },
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (OER)",
    "keyword": "oxygen evolution catalysts",
    "meta_data": "cobalt phosphate (Co-Pi) and nickel borate (Ni-B) oxygen evolution catalysts"
  },
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reac

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 760.96ms


📄 Gemini returned:
 ```json
[
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (OER)",
    "keyword": "oxygen evolution reaction",
    "meta_data": "high efficiency toward the oxygen evolution reaction (OER)"
  },
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (OER)",
    "keyword": "OER",
    "meta_data": "high efficiency toward the oxygen evolution reaction (OER)"
  },
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (O
⚠️ JSON parsing failed. Attempting fallback regex...
✅ Extracted 6 items via regex.
✅ Total items extracted for paper Electrospun Fe-Incorporated ZIF-67 Nanofibers for Effective Electrocatalytic Water Splitting, layer 'Reaction': 6
📄 Gemini returned:
 ```json
[
  {
    "layer": "Elemental Composition",
    "matched_category": "Earth-Abundant Elements",
    "keyword": "earth-abundant",
    "meta_data": "earth-abundant, noble metal-free"
  },
  {
    "layer": "Elemental Compos

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 739.61ms


📄 Gemini returned:
 ```json
[
  {
    "layer": "Process",
    "matched_category": "Water Electrolysis",
    "keyword": "water-splitting reaction",
    "meta_data": "electrocatalyst for the water-splitting reaction is essential"
  },
  {
    "layer": "Process",
    "matched_category": "Electrocatalysis",
    "keyword": "electrocatalyst",
    "meta_data": "Development of a highly efficient electrocatalyst"
  },
  {
    "layer": "Process",
    "matched_category": "Electrocatalysis",
    "keyword": "electrocatalytic ac
⚠️ JSON parsing failed. Attempting fallback regex...
✅ Extracted 12 items via regex.
✅ Total items extracted for paper Cobaltii ions and cobalt nanoparticle embedded porous organic polymers an efficient electrocatalyst, layer 'Process': 12
📄 Gemini returned:
 ```json
[
  {
    "layer": "Reaction",
    "matched_category": "Oxygen Evolution Reaction (OER)",
    "keyword": "OER",
    "meta_data": "standard OER and HER conditions"
  },
  {
    "layer": "Reaction",
    "matched_c

In [ ]:
# Step 7: Save extracted ontology to file
with open("updated_ontology.json", "w") as f:
    json.dump(ontology_extractions, f, indent=2)

print("✅ Saved updated ontology to 'updated_ontology.json'")

✅ Saved updated ontology to 'updated_ontology.json'


In [ ]:
import pandas as pd

# ✅ Save Semantic Scholar metadata to Excel
def export_semantic_scholar_metadata_to_excel(paper_list, output_file="semantic_scholar_papers.xlsx"):
    if not paper_list:
        print("⚠️ No papers to export.")
        return

    df = pd.DataFrame(paper_list)

    # Show preview
    print("📄 Exporting the following columns:", list(df.columns))
    print(f"✅ Total papers: {len(df)}")

    # Save to Excel
    df.to_excel(output_file, index=False)
    print(f"✅ Saved to '{output_file}'")

# ✅ Example call
export_semantic_scholar_metadata_to_excel(semantic_scholar_papers)


📄 Exporting the following columns: ['paper_id', 'title', 'abstract', 'authors', 'year', 'venue', 'url', 'doi']
✅ Total papers: 200
✅ Saved to 'semantic_scholar_papers.xlsx'


In [8]:
import json
from py2neo import Graph, Node, Relationship
from collections import defaultdict
import time

start_time = time.time()

# Load extracted ontology
with open("updated_ontology.json", "r") as f:
    ontology_extractions = json.load(f)

# Connect to Neo4j
neo4j_uri = "neo4j+s://ffdf4581.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_pass = "vLDWyCD-NJs6ejGPIpX5juhdorL5Q8jtvd0Z789bKwk"
graph = Graph(neo4j_uri, auth=(neo4j_user, neo4j_pass))

# Track uniqueness
unique_keywords = defaultdict(set)
unique_categories = set()
unique_layers = set()
paper_keywords = defaultdict(set)  # Track keywords per paper

# 🔵 Step 1: Create Layer, Category, Keyword, MetaData nodes
for layer, categories in ontology_extractions.items():
    if layer not in unique_layers:
        layer_node = Node("Layer", name=layer)
        graph.merge(layer_node, "Layer", "name")
        unique_layers.add(layer)

    for cat, items in categories.items():
        cat_key = f"{layer}|{cat}"
        if cat_key not in unique_categories:
            cat_node = Node("Category", name=cat, key=cat_key)
            graph.merge(cat_node, "Category", "key")
            graph.merge(Relationship(layer_node, "HAS_CATEGORY", cat_node))
            unique_categories.add(cat_key)

        for item in items:
            paper_id = item.get("paper_id", "unknown")
            for kw in item["keywords"]:
                kw_key = f"{layer}|{cat}|{kw}"
                if kw_key in unique_keywords[(layer, cat)]:
                    continue
                unique_keywords[(layer, cat)].add(kw_key)

                kw_node = Node("Keyword", name=kw, key=kw_key)
                graph.merge(kw_node, "Keyword", "key")

                cat_node = Node("Category", name=cat, key=cat_key)
                graph.merge(cat_node, "Category", "key")
                graph.merge(Relationship(cat_node, "HAS_KEYWORD", kw_node))

                for meta in item.get("parsed_meta", []):
                    meta_node = Node("MetaData", value=meta["value"], unit=meta["unit"], type=cat)
                    graph.create(meta_node)
                    graph.create(Relationship(kw_node, "HAS_METADATA", meta_node))

                paper_keywords[paper_id].add(kw_key)  # Collect keywords per paper

print("✅ Layer, Category, Keyword, and MetaData nodes created")

# 🔁 Step 2: Create Paper nodes and MENTIONS relationships
for paper_id, kw_keys in paper_keywords.items():
    paper_node = Node("Paper", id=paper_id)
    graph.merge(paper_node, "Paper", "id")
    for kw_key in kw_keys:
        kw_node = Node("Keyword", key=kw_key)
        graph.merge(kw_node, "Keyword", "key")
        rel = Relationship(kw_node, "MENTIONS", paper_node)
        graph.merge(rel)

print("✅ Paper nodes and MENTIONS relationships created")
result = sum(range(1_000_000))

end_time = time.time()
print(f"🎉 Knowledge graph construction completed in {end_time - start_time:.4f} seconds")

✅ Layer, Category, Keyword, and MetaData nodes created
✅ Paper nodes and MENTIONS relationships created
🎉 Knowledge graph construction completed in 1244.6653 seconds


In [ ]:
from openai import OpenAI
import json
import re
from collections import defaultdict
from itertools import combinations
import pandas as pd

# ✅ Load ontology
with open("updated_ontology.json", "r") as f:
    ontology_extractions = json.load(f)

# ✅ Build paper maps
category_to_papers = defaultdict(lambda: defaultdict(int))
paper_keyword_map = defaultdict(lambda: defaultdict(set))
kw_lookup = {}

for layer, cats in ontology_extractions.items():
    for cat, items in cats.items():
        for item in items:
            for kw in item["keywords"]:
                kw_lower = kw.lower()
                kw_lookup[kw_lower] = (layer, cat)
                paper_keyword_map[(layer, cat)][item["paper_id"]].add(kw_lower)
                category_to_papers[(layer, cat)][item["paper_id"]] += 1

# ✅ Gemini client
model = configure_gemini()


# ✅ Classify user query using Gemini + fallback
def classify_query_keywords(user_query, ontology):
    tokens = [t for t in re.findall(r"\w+", user_query.lower()) if len(t) > 1]

    ontology_summary = {layer: list(cats.keys()) for layer, cats in ontology.items()}
    prompt = f'''
You are a structured classification AI for materials science.
Ontology:
{json.dumps(ontology_summary, indent=2)}

Given the user query: "{user_query}"

Extract each meaningful keyword (ignore stopwords like 'and', 'for', etc).
Assign each keyword to the most relevant category and layer from the ontology.
Return only the results in valid JSON format:

[
  {{
    "keyword": "cheap",
    "category": "Earth-Abundant Elements",
    "layer": "Elemental Composition"
  }},
  ...
]
'''
    response = model.generate_content(prompt)
    content = response.text.strip().replace("```json", "").replace("```", "")


    try:
        model_output = json.loads(content)
    except json.JSONDecodeError:
        print("⚠️ Gemini output couldn't be parsed. Raw:\n", content)
        model_output = []

    categories = []
    found_set = set()
    model_keywords = set()

    for item in model_output:
        kw = item["keyword"].lower()
        model_keywords.add(kw)
        if item["layer"] != "Unknown" and item["category"] != "Unknown":
            if (item["layer"], item["category"]) not in found_set:
                categories.append((item["layer"], item["category"], kw))
                found_set.add((item["layer"], item["category"]))
                print(f"✅ Gemini: '{kw}' → {item['layer']} / {item['category']}")

    # Fallback for skipped tokens
    for tok in tokens:
        if tok not in model_keywords:
            if tok in kw_lookup:
                layer, cat = kw_lookup[tok]
                if (layer, cat) not in found_set:
                    categories.append((layer, cat, tok))
                    found_set.add((layer, cat))
                    print(f"🛟 Fallback: '{tok}' → {layer} / {cat}")
            else:
                print(f"❌ No match for: '{tok}'")

    return categories

# ✅ Enhanced filtering summary
def summarize_filtered_papers(filtered_out, query_keywords, opposites, paper_keyword_frequencies):
    rows = []

    for pid, info in filtered_out.items():
        title_text = (title_map.get(pid, "") or "").lower()
        abstract_text = (abstract_map.get(pid, "") or "").lower()

        for qk in query_keywords:
            matched_opp_keywords = []

            # Title matching
            title_rel = len(re.findall(rf'\b{re.escape(qk)}\b', title_text))
            title_opp = 0
            for opp in opposites.get(qk, []):
                count = len(re.findall(rf'\b{re.escape(opp)}\b', title_text))
                title_opp += count
                if count > 0:
                    matched_opp_keywords.append(opp)

            # Abstract matching
            abs_rel = len(re.findall(rf'\b{re.escape(qk)}\b', abstract_text))
            abs_opp = 0
            for opp in opposites.get(qk, []):
                count = len(re.findall(rf'\b{re.escape(opp)}\b', abstract_text))
                abs_opp += count
                if count > 0 and opp not in matched_opp_keywords:
                    matched_opp_keywords.append(opp)

            total_rel = title_rel + abs_rel
            total_opp = title_opp + abs_opp
            ratio = round(total_opp / total_rel, 2) if total_rel else float('inf')

            rows.append({
                "Paper ID": pid,
                "Query Keyword": qk,
                "Title Relevant Count": title_rel,
                "Title Opposing Count": title_opp,
                "Abstract Relevant Count": abs_rel,
                "Abstract Opposing Count": abs_opp,
                "Total Relevant Count": total_rel,
                "Total Opposing Count": total_opp,
                "Matched Opposing Keywords": ", ".join(sorted(set(matched_opp_keywords))),
                "Ratio": ratio,
                "Status": "Filtered" if ratio > info["threshold"] else "Kept"
            })

    return pd.DataFrame(rows)

def get_opposites_via_llm(keywords: list) -> dict:
    prompt = f"""
You are a materials science assistant.

For each of the following scientific keywords, return a list of their opposite or contradictory concepts in the context of electrocatalysis, water splitting, or electrochemical environments.

Keywords: {keywords}

Respond only in JSON format like this:
{{
  "acidic": ["alkaline", "basic"],
  "HER": ["OER", "oxygen evolution"],
  ...
}}
"""
    response = model.generate_content(prompt)
    content = response.text.strip().replace("```json", "").replace("```", "")

    try:
        return json.loads(content)
    except Exception:
        print("⚠️ Could not parse opposite keyword JSON. Raw output:\n", content)
        return {}

def get_synonyms_via_llm(keywords: list) -> dict:
    prompt = f"""
You are a materials science assistant.

For each of the following scientific keywords, return a list of their synonyms or semantically similar expressions used in electrocatalysis, water splitting, or electrochemistry papers.

Keywords: {keywords}

Respond only in JSON format like this:
{{
  "acidic": ["low pH", "acid media"],
  "HER": ["hydrogen evolution", "H2 generation"],
  ...
}}
"""
    response = model.generate_content(prompt)
    content = response.text.strip().replace("```json", "").replace("```", "")

    try:
        return json.loads(content)
    except Exception:
        print("⚠️ Could not parse synonym keyword JSON. Raw output:\n", content)
        return {}

def rank_by_pair_overlap_filtered(per_cat_hits, ranked_paper_ids):
    pair_paper_map = defaultdict(set)  # pid → set of category pairs
    category_pairs = list(combinations(per_cat_hits.keys(), 2))
    total_possible_pairs = len(category_pairs)

    for (cat1, cat2) in category_pairs:
        papers1 = set(per_cat_hits[cat1].keys())
        papers2 = set(per_cat_hits[cat2].keys())
        shared_papers = papers1 & papers2
        filtered_shared = shared_papers & ranked_paper_ids

        pair_label = f"{cat1[1]} ↔ {cat2[1]}"
        for pid in filtered_shared:
            pair_paper_map[pid].add(pair_label)

    ranked_by_pair_overlap = sorted(pair_paper_map.items(), key=lambda x: len(x[1]), reverse=True)
    return ranked_by_pair_overlap, total_possible_pairs

# ✅ Main pipeline
def find_common_papers(user_query, threshold=1.5):
    print(f"🔍 User query: {user_query}")
    categories = classify_query_keywords(user_query, ontology_extractions)

    if not categories:
        print("⚠️ No valid categories found.")
        return
    def print_relevant_keywords_per_paper(categories, paper_keyword_map):
        """
        Print all relevant keywords (and synonyms) that were matched per paper
        based on classified (layer, category) from the user query.
        """
        relevant_keywords_per_paper = {}

        # Extract the relevant (layer, category) pairs from the classified query
        relevant_layer_cats = [(layer, cat) for layer, cat, _ in categories]

        for layer_cat in relevant_layer_cats:
            paper_kw_map = paper_keyword_map.get(layer_cat, {})
            for paper_id, keywords in paper_kw_map.items():
                if paper_id not in relevant_keywords_per_paper:
                    relevant_keywords_per_paper[paper_id] = set()
                relevant_keywords_per_paper[paper_id].update(keywords)

        # Print results
        for pid, keywords in relevant_keywords_per_paper.items():
            print(f"📄 Paper ID: {pid}")
            print(f"🔑 Keywords: {sorted(keywords)}\n")

    # Example usage:
    print_relevant_keywords_per_paper(categories, paper_keyword_map)

    print(f"\n✅ Categories Used: {len(categories)}")
    for i, (layer, cat, kw) in enumerate(categories, 1):
        print(f"  {i}. '{kw}' → {layer} / {cat}")

    query_keywords = [kw.lower() for _, _, kw in categories]
    opposites = get_opposites_via_llm(query_keywords)
    synonyms = get_synonyms_via_llm(query_keywords)
    expanded_keywords = {kw: [kw] + synonyms.get(kw, []) for kw in query_keywords}


    print(f"\n🧪 Opposites used for filtering:")
    for qk in query_keywords:
        print(f"  - '{qk}': {opposites.get(qk, [])}")

    per_cat_hits = {}
    total_scores = defaultdict(int)
    paper_keyword_frequencies = defaultdict(lambda: defaultdict(int))
    filtered_out = {}

    for (layer, cat), hits in category_to_papers.items():
        for pid, count in hits.items():
            if (layer, cat) in [(l, c) for l, c, _ in categories]:
                total_scores[pid] += count
                per_cat_hits[(layer, cat)] = hits
            for kw in paper_keyword_map[(layer, cat)][pid]:
                paper_keyword_frequencies[pid][kw] += 1

    def is_dominated_by_opposites(paper_id):
        # Load and preprocess
        title_text = (title_map.get(paper_id, "") or "").lower()
        full_abstract = (abstract_map.get(paper_id, "") or "").lower()
        abstract_text = re.split(r'(references|refs?\.?:)', full_abstract)[0]

        title_flags = {}
        title_over_threshold = 0  # For low relevance sorting

        for qk in query_keywords:
            # Relevant keyword expansion
            relevant_terms = expanded_keywords.get(qk, [qk])
            title_rel = sum(len(re.findall(rf'\b{re.escape(term)}\b', title_text)) for term in relevant_terms)
            title_opp = sum(len(re.findall(rf'\b{re.escape(opp)}\b', title_text)) for opp in opposites.get(qk, []))

            if title_rel == 0 and title_opp == 0:
                title_flags[qk] = "missing"
            else:
                ratio = float('inf') if title_rel == 0 else title_opp / title_rel
                if ratio > 1:
                    title_flags[qk] = "fail"
                    title_over_threshold += 1
                elif ratio in [0, 1]:
                    title_flags[qk] = "pass"
                else:
                    title_flags[qk] = "other"

        # 🔴 Low Relevance: if any fail in title (ratio > 1)
        if "fail" in title_flags.values():
            filtered_out[paper_id] = {
                "reason": "title_fail",
                "title_flags": title_flags,
                "title_exceeded": title_over_threshold,
                "threshold": 1.0,
            }
            return True

        # 🟡 Moderate Relevance: at least one pass, one missing, no fails
        if "pass" in title_flags.values() and "missing" in title_flags.values():
            abstract_flags = []
            for qk in query_keywords:
                relevant_terms = expanded_keywords.get(qk, [qk])
                abs_rel = sum(len(re.findall(rf'\b{re.escape(term)}\b', abstract_text)) for term in relevant_terms)
                abs_opp = sum(len(re.findall(rf'\b{re.escape(opp)}\b', abstract_text)) for opp in opposites.get(qk, []))
                ratio = float('inf') if abs_rel == 0 else abs_opp / abs_rel
                abstract_flags.append(ratio)

            if all(r <= 1.5 for r in abstract_flags):
                return False  # Promote to high
            else:
                filtered_out[paper_id] = {
                    "reason": "moderate_abstract_check",
                    "abstract_ratios": abstract_flags,
                    "title_exceeded": 0,
                    "threshold": 1.5,
                }
                return False

        # 🟢 High Relevance: all pass
        if all(flag == "pass" for flag in title_flags.values()):
            return False

        # Default fallback
        return False


    # Apply filter: check every paper
    all_paper_ids = list(title_map.keys())
    ranked = []
    for pid in all_paper_ids:
        score = total_scores.get(pid, 0)
        if not is_dominated_by_opposites(pid):
            ranked.append((pid, score))
    ranked_paper_ids = {pid for pid, _ in ranked}


    # Show filtering report
    if filtered_out:
        summary_df = summarize_filtered_papers(filtered_out, query_keywords, opposites, paper_keyword_frequencies)

        try:
            from IPython.display import display
            display(summary_df)
        except:
            pass

        summary_df.to_csv("filtered_paper_breakdown.csv", index=False)

        print(f"\n🚫 Filtered out {len(filtered_out)} papers due to dominance of opposite keywords.")

    # Separate low relevance
    low_relevance = [(pid, info.get("title_exceeded", 0)) for pid, info in filtered_out.items() if info["reason"] == "title_fail"]
    low_sorted = sorted(low_relevance, key=lambda x: x[1])

    # Print ranked results
    print("\n📚 Ranked Papers:")
    print("\n🟢🟡 High & Moderate Relevance:")
    for i, (pid, score) in enumerate(sorted(ranked, key=lambda x: -x[1]), 1):
        print(f"  {i}. {pid} (mentions={score})")

    print("\n🔴 Low Relevance (sorted by number of over-threshold keywords):")
    for i, (pid, count) in enumerate(low_sorted, 1):
        print(f"  {i}. {pid} (keywords over threshold: {count})")

    # Filter hits to only ranked papers
    filtered_hits = {k: {pid: v for pid, v in d.items() if pid in ranked_paper_ids} for k, d in per_cat_hits.items()}

    if len(per_cat_hits) < 2:
        print("\n⚠️ Overlap analysis needs 2+ categories.")
        return

    # Run filtered overlap analysis
    ranked_by_pair_overlap, total_possible_pairs = rank_by_pair_overlap_filtered(filtered_hits, ranked_paper_ids)

    print(f"\n🏆 Full Ranking by Number of Category Pairs Shared (out of {total_possible_pairs} pairs):")
    for i, (pid, pair_set) in enumerate(ranked_by_pair_overlap, 1):
        pair_count = len(pair_set)
        print(f"  {i}. {pid} → shared in {pair_count}/{total_possible_pairs} pair(s)")
        print(f"     ⤷ Pairs: {sorted(pair_set)}")

    if not ranked_by_pair_overlap:
        print("⚠️ No multi-category-pair overlaps found.")

    import pandas as pd

    df_pairs = pd.DataFrame([
        {
            "Paper ID": pid,
            "Pair Count": len(pair_set),
            "Shared Pairs": ", ".join(sorted(pair_set))
        }
        for pid, pair_set in ranked_by_pair_overlap
    ])

    df_pairs.to_csv("shared_pair_ranked_papers.csv", index=False)
    print("✅ Exported shared papers to 'shared_pair_ranked_papers.csv'")


    print("\n🔄 Overlaps Between Categories:")
    for (l1, c1), (l2, c2) in combinations(per_cat_hits.keys(), 2):
        pids1 = set(paper_keyword_map[(l1, c1)].keys())
        pids2 = set(paper_keyword_map[(l2, c2)].keys())
        shared = pids1 & pids2
        filtered_shared = shared & ranked_paper_ids

        total_mentions = sum(
            len(paper_keyword_map[(l1, c1)][pid]) + len(paper_keyword_map[(l2, c2)][pid])
            for pid in filtered_shared
        )

        print(f"  • '{c1}' ↔ '{c2}': {len(filtered_shared)} shared papers, {total_mentions} keyword mentions in common")
        return ranked, low_sorted


# ✅ Run it
ranked, low_sorted = find_common_papers("acidic HER water splitting", threshold=1.5)

🔍 User query: acidic HER water splitting
✅ Gemini: 'acidic' → Environment / Acidic
✅ Gemini: 'her' → Reaction / Hydrogen Evolution Reaction (HER)
✅ Gemini: 'water splitting' → Process / Water Electrolysis
❌ No match for: 'water'
❌ No match for: 'splitting'
📄 Paper ID: Invited Atomic Layer Deposited Sulfides As PreCatalysts for Electrochemical Water Splitting
🔑 Keywords: ['0.5 m h2so4', 'acidic', 'acidic environment', 'ph 1', 'ph < 7', 'ph<', 'ph<7', 'ph=1']

📄 Paper ID: CoP nanoparticles anchored on NP-dual-doped graphene-like carbon as a catalyst for water splitting i
🔑 Keywords: ['1 m pbs', 'acidic', 'cell voltages', 'current density', 'electrolysis of water', 'h2 generation', 'her', 'hydrogen evolution', 'hydrogen evolution reaction', 'hydrogen from water', 'ph 1', 'ph < 7', 'ph<', 'ph<7', 'ph=1', 'reduction of protons', 'splitting h2o', 'water splitting']

📄 Paper ID: Synergistic niobium and manganese co-doping into RuO2 nanocrystal enables PEM water splitting under
🔑 Keywords: ['0

Paper ID    Query Keyword  \
0    Invited Atomic Layer Deposited Sulfides As Pre...           acidic   
1    Invited Atomic Layer Deposited Sulfides As Pre...              her   
2    Invited Atomic Layer Deposited Sulfides As Pre...  water splitting   
3    Polyoxometalate electrocatalysts based on eart...           acidic   
4    Polyoxometalate electrocatalysts based on eart...              her   
..                                                 ...              ...   
427  Co-Doped Ni3N Nanosheets with Electron Redistr...              her   
428  Co-Doped Ni3N Nanosheets with Electron Redistr...  water splitting   
429  Nanostructured materials on 3D nickel foam as ...           acidic   
430  Nanostructured materials on 3D nickel foam as ...              her   
431  Nanostructured materials on 3D nickel foam as ...  water splitting   

     Title Relevant Count  Title Opposing Count  Abstract Relevant Count  \
0                       0                     0                        1   
1                       0                     0                        5   
2                       1                     0                        1   
3                       1                     0                        0   
4                       0                     0                        0   
..                    ...                   ...                      ...   
427                     0                     0                        3   
428                     1                     0                        1   
429                     0                     0                        0   
430                     0                     0                        1   
431                     1                     0                        1   

     Abstract Opposing Count  Total Relevant Count  Total Opposing Count  \
0                          2                     1                     2   
1                          0                     5                     0   
2                          0                     2                     0   
3                          0                     1                     0   
4                          0                     0                     0   
..                       ...                   ...                   ...   
427                        0                     3                     0   
428                        0                     2                     0   
429                        0                     0                     0   
430                        0                     1                     0   
431                        0                     2                     0   

    Matched Opposing Keywords  Ratio    Status  
0                    alkaline    2.0  Filtered  
1                                0.0      Kept  
2                                0.0      Kept  
3                                0.0      Kept  
4                                inf  Filtered  
..                        ...    ...       ...  
427                              0.0      Kept  
428                              0.0      Kept  
429                              inf  Filtered  
430                              0.0      Kept  
431                              0.0      Kept  

[432 rows x 11 columns]


🚫 Filtered out 144 papers due to dominance of opposite keywords.

📚 Ranked Papers:

🟢🟡 High & Moderate Relevance:
  1. Digital Presentation Water Oxidation in Acid with Earth-Abundant Electrocatalysts (mentions=79)
  2. Bifunctional Noble-Metal Free Ternary Chalcogenide Electrocatalysts for Water Splitting (mentions=74)
  3. A membrane-free flow electrolyzer operating at high current density using earth-abundant catalysts f (mentions=61)
  4. Arming Ru with OxygenVacancyEnriched RuO2 SubNanometer Skin Activates Superior Bifunctionality for p (mentions=56)
  5. Sustainable oxygen evolution electrocatalysis in aqueous 1M H2SO4 with earth abundant nanostructured (mentions=53)
  6. Ru Single Atoms Anchored on Co3O4 Nanorods for Efficient Overall Water Splitting under pHUniversal C (mentions=48)
  7. Hydrogen evolution catalyzed by cobalt diimine-dioxime complexes (mentions=47)
  8. Engineering oxygen-evolving catalysts for acidic water electrolysis (mentions=45)
  9. Environmentally Frien

In [ ]:
# ✅ Post-processing: Rank by pair count first, then by relevant keyword mentions


# Load shared pairs
df_pairs = pd.read_csv("shared_pair_ranked_papers.csv")

# Build relevance level + score map
relevance_scores = {}
for pid, score in ranked:
    relevance_scores[pid] = ("high", score)
for pid, count in low_sorted:
    relevance_scores[pid] = ("low", 0)

# Assign scores
df_pairs["Relevance Level"] = df_pairs["Paper ID"].map(lambda x: relevance_scores.get(x, ("unknown", 0))[0])
df_pairs["Relevant Keyword Score"] = df_pairs["Paper ID"].map(lambda x: relevance_scores.get(x, ("unknown", 0))[1])

# Final sort: by pair count first, then relevant keyword mentions
df_pairs = df_pairs.sort_values(
    by=["Pair Count", "Relevant Keyword Score"],
    ascending=[False, False]
)

# Print top results
print("\n🏆 Shared Papers Ranked by Pair Count → Mentions:")
for i, row in df_pairs.iterrows():
    print(f"{row['Paper ID']} | Pairs: {row['Pair Count']} | Mentions: {row['Relevant Keyword Score']} | Relevance: {row['Relevance Level']}")

# Save by relevance group
for level in ["high", "moderate", "low"]:
    subset = df_pairs[df_pairs["Relevance Level"] == level]
    if not subset.empty:
        filename = f"shared_ranked_by_pairs_then_mentions_{level}.csv"
        subset.to_csv(filename, index=False)
        print(f"✅ Saved: {filename}")

# Optionally keep in memory
final_ranked_shared = df_pairs


🏆 Shared Papers Ranked by Pair Count → Mentions:
Digital Presentation Water Oxidation in Acid with Earth-Abundant Electrocatalysts | Pairs: 3 | Mentions: 79 | Relevance: high
Bifunctional Noble-Metal Free Ternary Chalcogenide Electrocatalysts for Water Splitting | Pairs: 3 | Mentions: 74 | Relevance: high
A membrane-free flow electrolyzer operating at high current density using earth-abundant catalysts f | Pairs: 3 | Mentions: 61 | Relevance: high
Arming Ru with OxygenVacancyEnriched RuO2 SubNanometer Skin Activates Superior Bifunctionality for p | Pairs: 3 | Mentions: 56 | Relevance: high
Sustainable oxygen evolution electrocatalysis in aqueous 1M H2SO4 with earth abundant nanostructured | Pairs: 3 | Mentions: 53 | Relevance: high
Ru Single Atoms Anchored on Co3O4 Nanorods for Efficient Overall Water Splitting under pHUniversal C | Pairs: 3 | Mentions: 48 | Relevance: high
Hydrogen evolution catalyzed by cobalt diimine-dioxime complexes | Pairs: 3 | Mentions: 47 | Relevance: high
Eng